# Actividad laboratorio: Estimación de parámetros de un activo financiero

## Objetivos

El objetivo de esta actividad es programar y así tener soltura a la hora de manejar datos financieros.

## Pautas de elaboración

En uno de los recursos audiovisuales se ha explicado como ir a una base de datos donde aparecen las cotizaciones de las grandes empresas en el IBEX-35. En primer lugar, se recomienda considerar un activo, descargarse los datos y ajustar los parámetros del modelo log-normal utilizando dos de los tres métodos de estimación que se explican en el Tema 6.

## Parte 1: Definición de las funciones que vamos a utilizar y carga de librerías

En python hay multitud de librerías que podemos utilizar, en este caso utilizaremos librerías standar para el analísis de datos y acceder a la base de datos de Yahoo Finance.
Además, vamos a definir una serie de funciones para proceder a ajustar un modelo log-normal.

El modelo Log-Normal se puede describir matemáticamente como:
$$ 
    \begin{cases}
    dS(t) = \mu S(t) + \sigma S(t)dW(t)\ \ donde \  \mu \in \mathbb{R} \ y \ \sigma >0, \\
    S(0) = s_0
    \end{cases} 
$$
Aplicando el Lema de Ito y resolviendo la EDE, obtenemos que $S(t) = s_0 e^{(\mu - \frac{1}{2}\sigma ^2)t + \sigma W(t) }$

### Librerías

In [28]:
#pip install yfinance   -> Con esta librería podemos conectarnos a los datos de Yahoo Finance.
#pip install pandas     -> Librería por excelencia para el manejo de datos con python.
import yfinance as yf  
import pandas as pd 
import numpy as np
from math import exp

### Funciones

####  Media

Aplicando el operador esperanza se obtiene la siguiente expresión: $$ \mathbb{E}[S(t)] = s_0 \cdot e^{\mu t}, $$ Esto lo podemos programar de la siguiente manera:

In [41]:
def media(datos,t = None):
    """
    La siguiente función nos devuelve el media del modelo Log-Normal en el instate t:
    Input:
        - datos(List[Number])   : Datos a analizar.
        - t (Number)      : Vector de tiempos o el instante t. (Valor opcional, si no se rellena, devuelve en cada instante de t)
    Output:
        - eS(List[Number])  : Vector de medias de la solución en todos los instantes o media de la solución en el instante t.
    """
    from math import exp
    mu = 1/len(datos)*sum(datos)
    s0 = datos[0]
    eS = []
    if t:
        eS.append(s0*exp(mu*t))
    else:
        for t in range(len(datos)):
            eS.append(s0*exp(mu*(t)))
    return(eS)

media(prueba["Open"])

[5.197000026702881,
 987.80118081847,
 187752.77425684506,
 35686436.628809765,
 6782971725.999026]

Aplicando el operador esperanza se obtiene la siguiente expresión: $$ \mathbb{V}[S(t)] = \mathbb{E}[S(t)^2] - \mathbb{E}[S(t)]^2 = s_0^2 \cdot e^{2 \mu t} \cdot (e^{\sigma^2 t} - 1), $$ Esto lo podemos programar de la siguiente manera:

In [46]:
def desvTipica(datos,t = None):
    """
    La siguiente función nos devuelve la desviación típica del modelo Log-Normal en el instate t:
    Input:
        - datos(List[Number])   : Datos a analizar.
        - t (Number)      : Vector de tiempos o el instante t. (Valor opcional, si no se rellena, devuelve en cada instante de t)
    Output:
        - eS(List[Number])  : Vector desviación típica la solución en todos los instantes o desviación típica de la solución en el instante t.
    """
    from math import exp, sqrt
    import numpy as np 
    mu = np.mean(datos)
    sigma2 = np.std(datos)**2
    s0 = datos[0]
    eS = []
    if t:
        eS.append(sqrt((s0**2)*exp(2*mu*t)*(exp(sigma2*t)-1)))
    else:
        for t in range(len(datos)):
            eS.append(sqrt((s0**2)*exp(2*mu*t)*(exp(sigma2*t)-1)))
    return(eS)

desvTipica(prueba["Open"])

[0.0,
 31.830742856011288,
 8558.374860872009,
 1992812.7206533635,
 437487276.5091437]

## Parte 2: Análisis exploratorio de los datos

En esta segunda parte, vamos a analizar de forma libiana los datos y en caso de necesitar alguna transformación para poder aplicar nuestro modelo procederemos con ello.

In [37]:
simbolo = "BBVA.MC"
bbva = yf.Ticker(simbolo)
bbva.info["longBusinessSummary"] + " " + bbva.info["website"]

bbva.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-03 00:00:00+01:00,5.015342,5.039979,4.980147,4.990705,8244257,0.0,0
2000-01-04 00:00:00+01:00,4.927352,4.959028,4.842883,4.860481,8522096,0.0,0
2000-01-05 00:00:00+01:00,4.814727,4.839365,4.730259,4.751376,12159826,0.0,0
2000-01-06 00:00:00+01:00,4.751376,4.751376,4.751376,4.751376,0,0.0,0
2000-01-07 00:00:00+01:00,4.853442,4.853442,4.761934,4.811207,62261944,0.0,0
...,...,...,...,...,...,...,...
2022-10-28 00:00:00+02:00,5.197000,5.248000,5.021000,5.246000,20870488,0.0,0
2022-10-31 00:00:00+01:00,5.240000,5.272000,5.172000,5.209000,15559086,0.0,0
2022-11-01 00:00:00+01:00,5.249000,5.299000,5.229000,5.255000,14233887,0.0,0
